# PART 3 - Machine Learning models
#### Dans cette partie, nous allons diviser nos données, créer des features (OneHot, TF-IDF) et construire nos modèles (Baselines et modèles améliorés)

In [1]:
from preprocessing import Preprocess, split_data
from utils import display_score
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
parquet_data_path = "../data/"
preprocess = Preprocess()
df = preprocess.create_dataframe(parquet_data_path, preprocess=True)

part-00004-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00003-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00001-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00002-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00000-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet


In [3]:
df

,target,day,sous_domaine,domaine,top_domaine,tokens_path,100,1000,1001,1002,...,990,991,992,993,994,995,996,997,998,999
0,"[329, 1234, 5183, 96, 377]",4,www,societe,com,societ madam karin pinchon,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[158, 650, 1175, 831, 953]",4,www,ebay-kleinanzeigen,de,nu fbaumstamm,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[325, 253, 1775, 640, 543]",4,psychologie,aufeminin,com,forum copain troubl comport fd,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[1143, 210, 531, 18, 41]",4,fr.shopping,rakuten,com,poweron pr,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[1171, 1071, 1192, 1533, 1277]",4,www,cdiscount,com,search coqu samsung galaxy,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67590,"[1193, 318, 22, 1107, 1111]",12,www,senscritique,com,ser hard critiqu,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67591,"[997, 1598, 993, 992, 984]",13,www,cdiscount,com,pret port vet blaz femm top manch longu vest f...,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
67592,"[1193, 1410, 1049, 1187, 358]",13,www,programme-tv,net,programm autr laventur robinson laventur robinson,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67593,"[63, 5184, 1254, 1119]",12,fr,windfinder,com,forecast point roug,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Trois approches peuvent etre prises pour la colonne sous_domaine:
- La prétraiter (enlever les www et les lettres, tokenizer...) et l'ajouter à la colonne path pour former une description
- L'enlever 
- La prétraiter afin de construire une feature catégorielle.

Pour l'instant nous allons l'enlever

## Construction des features à partir du path
Différentes approches peuvent etre utilisées sur la colonne path:
#### L'utilisation de TF-IDF 
- TF-IDF: refléte l'importance d'un mot pour un document dans une collection (corpus) mais ne prend pas en compte le sens sémantique des mots. TF signifie la probabilité d'occurrence d'un mot dans une phrase.
- TF-IDF donne plus d'importance aux mots qui apparaissent moins fréquemment dans l'ensemble du corpus et donne également de l'importance aux mots les plus fréquents qui apparaissent dans chaque donnée.

Nous testerons les featurizations suivantes:
- TF-IDF: unigrams, bigrams, trigrams et word n-grams.
- TF-IDF based character: unigrams, bigrams, trigrams. (considérer une séquence de caractères plutôt qu'une séquence de mots)


#### L'utilisation des Embeddings (word2vec)
- Word2vec est l'un des modèle de l'état de l'art pour les embeddings.  Il permet de convertir du texte en vecteurs numériques tout en préservantles relations sémantiques entre les mots.
- vecteur de 300 dimensions 

#### L'utilisation d'une combinaison de TF-IDF et moyenne des Embeddings:
Nombiner les n-grammes de caractères avec les vecteurs obtenus avec Word2Vec

In [4]:
X_train, X_test, y_train, y_test, target_train, target_test = split_data(df, test_size=0.25)
X_train.shape

(50696, 6)
(16899, 6)
Index(['day', 'domaine', 'top_domaine', 'tokens_path'], dtype='object')
Index(['day', 'domaine', 'top_domaine', 'tokens_path'], dtype='object')


(50696, 9216)

In [5]:
y_train.shape

(50696, 1903)

## Réduction de la dimension des labels
Nous avons 1903 labels pour notre target ce qui reprensente un nombre assez elevé pour de la classification multi-label.
Nous allons donc effectuer une réduction sur l'espace de dimension des labels.
Plusieurs algorithmes existent pour la réduction de l'espace des labels: compressed sensing (CS) et la  Principal Label Space Transformation (PLST) qui est l'équivalente du PCA sur les features. Egalement des techniques d'embeddings entre targets
Contrairement à ces techniques qui sont indépendentes de nos entrées, il en existe aussi d'autres qui le sont comme par exemple la CPLST ( Conditional Principal Label Space Transformation ) 

#### PLST :
PLST or Principal Label Space Transformation est une méthode de réduction de dimensionnalité utilisée explicitement pour les labels des datasets et permettant de les représenter géométriquement tout en minimisant l'erreur de reconstruction, nous combinons cette méthode ensuite avec des baselines modèles pour l'apprentissage.
  

Un autre moyen serait d'essayer d'utiliser une approches par embedding sur les labels afin d'extraire les labels qui sont souvent ensemble.

#### Label Graph :
C'est une méthode permettant de représenter un embedding des différents labels de l'output en créant ainsi un graphe de lien représentations des interactions continues entre les différents labels attendues. C'est un modèle qui a démontré dans la littérature une accuracy élevé et des résultats très concluants dans le domaine de la classification multi-label.


Pour la classification multi-label, il existe une librarie nommée `scikit-multilearn` qui s'appuie sur la librarie scikit-learn. Elle contient aussi un wrapper autour de  MEKA qui propose une implémentation des méthodes d'apprentissage et d'évaluation multi-labels comme la PLST et le LABELGRaph

### Création des modèles :
#### Baseslines: 
- Dans un premier temps, nous allons construire nos modèles Baselines avec l'ensemble des targets présentes dans notre dataset. Chaque modèle essaiera de prédire les targets sur soit le nombre total de 1009 targets ( Ce qui n'est peut etre pas une bonne approche), ou sur la réduction de dimension des 1009 targets. 
- Après avoir construit les baslines models, nous essaierons d'améliorer les modèles avec des hyperparamètres pour voir s'il y a une augmentation du score F1.

#### Vu le peu de temps que j'ai en soirée après la journée au stage, j'ai préféré au lieu de me focaliser sur l'amélioration d'une seule approche, me documenter sur d'autres approches parallelement à l'entrainement des modèles et essayer de les implementer.

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import (KNeighborsClassifier,
                               NeighborhoodComponentsAnalysis)


### Multi-label classification
#### 1.  OneVsRest: 
Le problème est décomposé en un problème de classification binaire multiple. Nous choisissons une classe et formons un classificateur binaire avec les échantillons de la classe sélectionnée d'un côté et tous les autres échantillons de l'autre côté. Ainsi, nous obtiendrons N classificateurs pour N étiquettes et lors du test nous classerons simplement l'échantillon comme appartenant à la classe avec le score maximum parmi les N classificateurs.

#### Sans réduction de dimension des labels: 

In [7]:
classifier1 = OneVsRestClassifier(LogisticRegression(penalty='l1', solver='liblinear', class_weight="balanced"), n_jobs=-1)
classifier1.fit(X_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                 penalty='l1',
                                                 solver='liblinear'),
                    n_jobs=-1)

In [9]:
predictions_train = classifier1.predict(X_train)
predictions = classifier1.predict(X_test)

In [10]:
pickle.dump(classifier1, open("finalized_model.sav", 'wb'))

In [11]:
classifier1 = pickle.load(open("finalized_model.sav", 'rb'))

In [12]:
print("train score")
display_score(y_train, predictions_train)
print("test score")
display_score(y_test, predictions)

train score

Micro-average quality numbers
Precision: 0.3107, Recall: 0.9978, F1-measure: 0.4738


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)



Macro-average quality numbers
Precision: 0.3310, Recall: 0.9760, F1-measure: 0.4681

Classification Report
test score

Micro-average quality numbers
Precision: 0.2285, Recall: 0.7228, F1-measure: 0.3472

Macro-average quality numbers
Precision: 0.1373, Recall: 0.3925, F1-measure: 0.1916

Classification Report


#### Avec réduction de dimension PCA: 
La réduction de l'espace des labels avec la PCA n'est pas la méthodes la plus appropriée car nous avons une matrice sparse et il est préférable d'appliquer la TruncatedSVD sur la les labels. Cependant, nous pourront testerons la PCA et verrons ce que ça donnera

In [16]:
from sklearn.decomposition import PCA

pca = PCA(n_components=300, random_state=42)
pca.fit(y_train)

print(pca.explained_variance_ratio_.sum())

0.7794021262243102


In [17]:
y_train_pca = pca.transform(y_train)
y_test_pca = pca.transform(y_test)

In [ ]:
classifier1 = OneVsRestClassifier(LogisticRegression(penalty='l1', solver='liblinear', class_weight="balanced"), n_jobs=-1)
classifier1.fit(X_train, y_train)


In [ ]:
predictions_train = classifier1.predict(X_train)
predictions = classifier1.predict(X_test)
print("train score")
display_score(y_train, predictions_train)
print("test score")
display_score(y_test, predictions)

#### Reduction de dimension TruncatedSVD

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(y_train)
print(svd.explained_variance_ratio_.sum())

y_train_svd = pca.transform(y_train)
y_test_svd = pca.transform(y_test)

#### 2.Binary Relevance:
Transformer un problème multi-label avec K étiquettes en des problèmes de classification binaire séparés. Chaque classificateur prédit si une étiquette est présente ou non.
Cette technique ignore les relations entre les étiquettes.

Les deux techniques présentées en haut traitent le problème multi-label (choix multiples) en une série de questions oui/non.

In [12]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())
classifier.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
pickle.dump(classifier, open("finalized_model.sav", 'wb'))

In [ ]:
classifier1 = pickle.load(open("finalized_model.sav", 'rb'))

In [ ]:
predictions = classifier.predict(X_test)
predictions_train = classifier.predict(X_train)
accuracy_score(y_test,predictions)
print('AUC ROC is {}'.format(roc_auc_score(y_test,predictions.toarray())))
from sklearn.metrics import roc_auc_score
print("train score")
display_score(y_train, predictions_train)
print("test score")
display_score(y_test, predictions)

#### 3.Classifier Chains:
Un classificateur 1 sera formé sur les données d'entrée. La sortie du classificateur 1 sera alimentée en entrée pour le classificateur 2, qui prédit la deuxième étiquette, la sortie du classificateur 2 sera alimentée en entrée pour le classificateur 3 et ainsi de suite.

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

classifier = ClassifierChain(LogisticRegression())
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

print('AUC ROC is {}'.format(roc_auc_score(y_test,predictions.toarray())))